#knn

* knn은 lazy learning이라고 한다. eager learning의 반대되는 개념. 즉, 학습이 늦추어져 있다가 질의가 일어나는 시점에 일어남.
* knn은 매우 단순하지만 정확성이 비교적 높음.
* 입력데이터가 과다할 경우, 테스트데이터의 비교가 비효율화

* 예를 들어, 사람마다 좋아하는 과일이 다른 경우, 

■ Number of frequent flyer miles earned per year
■ Percentage of time spent playing video games
■ Liters of ice cream consumed per week

| 비행기 마일리지 | 비디오게임하는 시간 | 아이스크림 소비량 | 이상형정도 |
|-------------|----------------|---------------|------|
|2.0|3.0|2.0| 1|
|5.0|3.0|2.0| 2|





* 단위가 다른 변수는 정규화 (최소, 최대가 0~1이 되도록 변환하거나 z값을 계산하여 사용함)
* 명목변수는 0,1로 변환

## algorithm knn
* 학습단계에서는 학습데이터 (자질(features), 클래스(class label)로 이루어진 벡터)를 준비해 놓음. 준비만 하고 모델링, 연산 등은 하지 않음.
* 분류단계에서는 테스트데이터를 학습데이터 가운데 가장 가까운 k개를 골라서, 가장 많은 클래스로 투표하듯이 분류함 (majority voting)

knn알고리듬의 입력데이터는 학습데이터(S), 테스트데이터(x), 투표갯수(k). 결과는 결정된 클래스.
* input: S,x,k
    * S: training dataset m(n_features) x n(n_samples)
    * x: test instance
    * c: class label
    * k: the number of most similar examples in S 
* output: class of x majority class

절차
* for (feature,class) $\in$ S do 
    * compute the distance d(feature,x)
* end for
* sort d by increasing order
* count the number of occurences of each class among the k nearest neighbors
* Assign d the class that is the most frequent class (or the majority class).


알고리듬에 따른 개발
1. data read - 데이터 준비
2. distance - 학습데이터와 테스트데이터 거리(유사도) 계산
3. select k neighbors - 가장 가까운 k개 선정
4. decision - k개에서 voting하여 결정
5. accuracy - 정확성

## 1. data read

In [90]:
#디렉토리 여러 파일을 읽는 실습.
import os
import numpy as np
# git clone https://github.com/pbharrin/machinelearninginaction.git
# cd Ch02/trainingDigits; unzip digits.zip
dir=os.getenv('HOME')+'/Code/git/else/machinelearninginaction/Ch02/trainingDigits'
#print dir
os.chdir(dir)
trainingFileList=os.listdir(dir)
filename=trainingFileList[1]
print "file name =",filename
#파일명 첫 글자 (underscore '_'로 분리해서)가 클래스명
print "class is ",filename.split('_')[0]

file name = 0_1.txt
class is  0


In [91]:
#파일 너비32, 높이32를 읽어 리스트를 만듦.
#입력: 파일명, 출력: 리스트
def img2vector(filename):
    height=32;
    width=32;
    mylist = []
    fr = open(filename)
    for i in range(height):
        lineStr=fr.readline()
        #print len(mylist)
        for j in range(width):
            mylist.append(int(lineStr[j]))
    #mat = numpy.array(mylist)
    return mylist

### 벡터를 합하는 문제
* img2vector는 한 파일에서 읽음.
* 아래 32x32 (=1024)형식, 1934개 파일의 데이터를 읽어 하나의 벡터로 합침.
* 아래를 보면 0이 보임. 즉, 파일명의 맨 앞 글자 ('_'로 분리한 결과)

In [92]:
!cat 0_114.txt

In [93]:
data=img2vector(filename)
print "saved 32 x 32 = ",len(data)

#여러 파일을 읽을 경우, 벡터를 만드는 연습.
a=np.array(range(0,10))
b=np.array(range(10,20))
ab=np.vstack([a,b])
print "a+b=",ab.shape
c=np.array(range(30,40))
abc=np.vstack([ab,c])
print "3x10테이블모양의 a+b+c=",abc

saved 32 x 32 =  1024
a+b= (2, 10)
3x10테이블모양의 a+b+c= [[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [30 31 32 33 34 35 36 37 38 39]]


In [94]:
# 디렉토리에 있는 파일을 읽어서 벡터로 만듦.
m=len(trainingFileList)
print m
trainingMat=np.zeros((m,1024))
hwlabels=[]
filenames=[]
for i in range(m):
    filename=trainingFileList[i]
    trainingMat[i,:]=img2vector(filename)
    classNumStr=int(filename.split('_')[0])
    hwlabels.append(classNumStr)
    filenames.append(filename)
#출력형식(formatted print)을 보기좋게
print "trainingMat{0}".format( trainingMat.shape)
print trainingMat[:5,:15]
print "hwlabels %d" % len(hwlabels)
print "hwlabels 한 줄에 1934개이지만 5개만 출력하면 {0}".format(hwlabels[:5])

1934
trainingMat(1934, 1024)
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]
hwlabels 1934
hwlabels 한 줄에 1934개이지만 5개만 출력하면 [0, 0, 0, 0, 0]


## 2. distance
### Euclidean distance

$$\|x\| := \sqrt{x_1^2 + \cdots + x_n^2}$$


In [95]:
import numpy as np
a=np.array([5,2,3,1,4])
b=np.array([2,4,3,4,5])
print np.linalg.norm(a-b)

#벡터연산을 하지 않는 경우
print np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2 + (a[2]-b[2])**2 +(a[3]-b[3])**2 +(a[4]-b[4])**2)
#어느 연산이 시간이 적게 걸리는지
import timeit
%timeit np.linalg.norm(a-b)
%timeit np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2 + (a[2]-b[2])**2 +(a[3]-b[3])**2 +(a[4]-b[4])**2)

4.79583152331
4.79583152331
The slowest run took 6.17 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 14.9 µs per loop
100000 loops, best of 3: 12.9 µs per loop


In [96]:
import numpy as np
#group = np.array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
g=np.array([[1.0,2.0],[2.0,3.0],[0,0],[4.0,5.0]])
#클래스c를 저장하는 associative array
c=np.array(['C','B','A','B'])
x=np.array([0,0])
print [np.linalg.norm(i-x) for i in g]
print ((g-x)**2).sum(axis=1)
print np.sqrt(((g-x)**2).sum(axis=1))
distances=[np.linalg.norm(i-x) for i in g]

[2.2360679774997898, 3.6055512754639891, 0.0, 6.4031242374328485]
[  5.  13.   0.  41.]
[ 2.23606798  3.60555128  0.          6.40312424]


## 3. select k neighbors

In [97]:
sortedDistIndices=np.array(distances).argsort()
#2번째 0, 0번째 2.23, 1번째 3.60, 3번째 6.40
print sortedDistIndices
#g를 x와 가장 가까운 것부터 출력하면
print [g[i] for i in sortedDistIndices]
print [c[i] for i in sortedDistIndices]
#argsort의 실습.
#3은 3번째인 '2', 2는 2번째인 '3', 0은 0번째인 '5', 1은 1번째 '7'
x = np.array([5, 7, 3, 2])
print np.argsort(x)

#앞의 k개를 sorting하여 가장 많은 빈도수 있는 것을 고름
k=3
print np.argsort(x)[:k]

[2 0 1 3]
[array([ 0.,  0.]), array([ 1.,  2.]), array([ 2.,  3.]), array([ 4.,  5.])]
['A', 'C', 'B', 'B']
[3 2 0 1]
[3 2 0]


## 4. decision

* Python에서 제공하는 Set기능을 사용하여 빈도가 가장 높은 요소를 선정.
* most_common함수는 빈도가 가장 높은 요소와 빈도수를 내림차순으로 출력

In [98]:
#혹은 collections를 사용
import collections
a1=['a','b','a','c']
a2=[1,2,3,4,4,4,4,2,42,5]
print collections.Counter(a1).most_common()

#이렇게 해도 됨.
labelcount={}
for i in range(len(a1)):
    l=a1[i]
    if l not in labelcount:
        labelcount[l]=1
    else:
        labelcount[l]+=1
import collections
print "labelcount=",labelcount
#labelcount에서 가장빈도가 가장 높은 요소 돌려줌. 즉 a는 2회.
print "most frequent item =",collections.Counter(labelcount).most_common(1)

#g=group
#np.sqrt((g[0]-x[0])**2 + (g[1]-b[1])**2 + (a[2]-b[2])**2 +(a[3]-b[3])**2 +(a[4]-b[4])**2)
#def classify(x,S,k):
    

[('a', 2), ('c', 1), ('b', 1)]
labelcount= {'a': 2, 'c': 1, 'b': 1}
most frequent item = [('a', 2)]


## 알고리듬 (위 2,3,4) 묶어서 하나의 함수로

In [106]:
#x:테스트하려는 데이터 (features)
#S: DataSet (feature의 집합)
#c: class
#k: 몇 개 가까운 이웃??
def classify0(x, S, c, k):
  import collections
  distances=[np.linalg.norm(i-x) for i in S]
  sortedDistIndices=np.array(distances).argsort()
  #print [S[i] for i in sortedDistIndices]
  #print [c[i] for i in sortedDistIndices]
  knn=[]
  for i in range(k):
        j=sortedDistIndices[i]
        knn.append(c[j])
        print "\tindex {0} traindata {1} class {2}".format(j,S[j][:5],c[j])
  majority=collections.Counter(knn).most_common(1)
  print "\tmajor class는 {0}".format(majority)

#x, g, c, k=1
print "g=",g
print "c=",c
x=[0,0]
print "now classifying..."
print "\n입력데이터{0}가운데 {1}과 가장 가까운 데이터와 클래스를 순서대로 나열하면".format(g, x)
classify0([0,0],g,c,3)
#[(0,1)]의 의미는 가장 가까운(most_common)은 0번째 즉 (0,0)이 1회 발생

g= [[ 1.  2.]
 [ 2.  3.]
 [ 0.  0.]
 [ 4.  5.]]
c= ['C' 'B' 'A' 'B']
now classifying...

입력데이터[[ 1.  2.]
 [ 2.  3.]
 [ 0.  0.]
 [ 4.  5.]]가운데 [0, 0]과 가장 가까운 데이터와 클래스를 순서대로 나열하면
	index 2 traindata [ 0.  0.] class A
	index 0 traindata [ 1.  2.] class C
	index 1 traindata [ 2.  3.] class B
	major class는 [('A', 1)]


In [107]:
#데이터 10개 정도만 가지고 실습.
#numpy.version.version 1.9
classify0(trainingMat[500],trainingMat,hwlabels,5)
print "{0} {1}".format(filenames[500],filenames[519])

	index 500 traindata [ 0.  0.  0.  0.  0.] class 2
	index 568 traindata [ 0.  0.  0.  0.  0.] class 2
	index 414 traindata [ 0.  0.  0.  0.  0.] class 2
	index 569 traindata [ 0.  0.  0.  0.  0.] class 2
	index 519 traindata [ 0.  0.  0.  0.  0.] class 2
	major class는 [(2, 5)]
2_25.txt 2_42.txt
